In [75]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np

In [121]:
out_df = pd.DataFrame(columns=['Year', 'Total', 'formattedAddress'])

In [122]:
pop = pd.read_csv('population.csv',sep=',')
grouped = pop.groupby('formattedAddress')

In [123]:
for county, group in grouped:
    ada = grouped.get_group(county)
    data = ada[['Year', 'Total']].dropna().astype('float')
    model = smf.ols(formula="Total ~ Year", data=data)
    result = model.fit()
    future_years = pd.DataFrame(np.arange(2016,2026).astype('float'), columns=['Year'])
    sm.add_constant(future_years)
    new_data = pd.DataFrame({ 'Total': result.predict(future_years), 'Year': future_years['Year'],
               'formattedAddress': np.repeat(county, 10)  })
    existing_data = pd.DataFrame({'Total': data['Total'], 'Year': data['Year'],
                              'formattedAddress': np.repeat(county, data['Year'].size)})
    out_df = out_df.append(existing_data)
    out_df = out_df.append(new_data)

In [126]:
out_df.to_csv('predicted_population.csv', columns=['Total','Year','formattedAddress'], index=False)

In [190]:
water = pd.read_csv('water.csv',sep=',', dtype={'ModFIPS': 'object', 'FIPS': 'int'})

In [177]:
grouped = water.groupby('FIPS')
out_df = pd.DataFrame(columns=['Water', 'Year', 'FIPS'])
missing = 0
for county, ada in grouped:
    ada = grouped.get_group(county)
    data = ada[['YEAR', 'TO-WTotl']].dropna().astype('float')
    data.columns = ['Year', 'Water']
    model = smf.ols(formula="Water ~ Year", data=data)
    result = model.fit()
    future_years = pd.DataFrame(np.arange(2016,2026).astype('float'), columns=['Year'])
    sm.add_constant(future_years)
    new_data = pd.DataFrame({ 'Water': result.predict(future_years), 'Year': future_years['Year'],
           'FIPS': np.repeat(county.astype('int'), 10)  })
    w = np.array(data['Water'])
    y = np.array(data['Year'])
    nan = np.nan
    if w.size != 3:
        missing = missing + 1
        continue
        
    existing_data = pd.DataFrame({
        'Water': [w[0], nan, nan, nan, nan, w[1], nan, nan, nan, nan, w[2]] },
                             index=[y[0], 2001, 2002, 2003, 2004, y[1], 2006, 2007, 2008, 2009, y[2]])
    existing_data = existing_data.interpolate(method='index')
    existing_data = pd.DataFrame({ 'Water': existing_data['Water'], 'Year': existing_data.index,
                             'FIPS': np.repeat(county.astype('int'), 11) })
    out_df = out_df.append(existing_data)
    out_df = out_df.append(new_data)

In [182]:
out_df['FIPS'] = out_df['FIPS'].astype('int')
out_df['Year'] = out_df['Year'].astype('int')

In [184]:
mod_fips = pd.DataFrame({'FIPS': water['FIPS'], 'ModFIPS': water['ModFIPS']})
out_df.join()

,STATE,STATEFIPS,ModFIPS,COUNTYFIPS,FIPS,YEAR,TP-TotPop,TO-WGWFr,TO-WGWSa,TO-WGWTo,TO-WSWFr,TO-WSWSa,TO-WSWTo,TO-WFrTo,TO-WSaTo,TO-WTotl
0,AL,1,1001,1,1001,2000,43.670,10.65,0.0,10.65,26.60,0.00,26.60,37.25,0.00,37.25
1,AL,1,1003,3,1003,2000,140.420,27.04,0.0,27.04,7.59,0.00,7.59,34.63,0.00,34.63
2,AL,1,1005,5,1005,2000,29.040,6.36,0.0,6.36,1.06,0.00,1.06,7.42,0.00,7.42
3,AL,1,1007,7,1007,2000,20.830,4.00,0.0,4.00,0.00,0.00,0.00,4.00,0.00,4.00
4,AL,1,1009,9,1009,2000,51.020,2.93,0.0,2.93,29.33,0.00,29.33,32.26,0.00,32.26
5,AL,1,1011,11,1011,2000,11.710,2.59,0.0,2.59,0.55,0.00,0.55,3.14,0.00,3.14
6,AL,1,1013,13,1013,2000,21.400,3.91,0.0,3.91,0.00,0.00,0.00,3.91,0.00,3.91
7,AL,1,1015,15,1015,2000,112.250,21.05,0.0,21.05,3.75,0.00,3.75,24.80,0.00,24.80
8,AL,1,1017,17,1017,2000,36.580,0.28,0.0,0.28,11.98,0.00,11.98,12.26,0.00,12.26
9,AL,1,1019,19,1019,2000,23.990,1.39,0.0,1.39,4.85,0.00,4.85,6.24,0.00,6.24


In [183]:
out_df.to_csv('predicted_water.csv', columns=['Water','Year','FIPS'], index=False)
